Download the Brazilian coins dataset, adapted from [Kaggle -Ronaldo S Moura](https://www.kaggle.com/ronaldosm96/brazilian-coins-dataset) as a split in subdirectories for the coin classes.

In [ ]:
!wget https://edshare.gcu.ac.uk/id/document/61314 \
      -O /content/Brazilian_coins_dataset_ClassSplit.zip

--2025-03-25 20:48:26--  https://edshare.gcu.ac.uk/id/document/61314
Resolving edshare.gcu.ac.uk (edshare.gcu.ac.uk)... 46.22.140.159
Connecting to edshare.gcu.ac.uk (edshare.gcu.ac.uk)|46.22.140.159|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://edshare.gcu.ac.uk/9958/1/Brazilian_coins_dataset_ClassSplit.zip [following]
--2025-03-25 20:48:27--  https://edshare.gcu.ac.uk/9958/1/Brazilian_coins_dataset_ClassSplit.zip
Reusing existing connection to edshare.gcu.ac.uk:443.
HTTP request sent, awaiting response... 200 OK
Length: 33540668 (32M) [application/zip]
Saving to: ‘/content/Brazilian_coins_dataset_ClassSplit.zip’

/content/Brazilian_ 100%[===================>]  31.99M  10.7MB/s    in 3.0s    

2025-03-25 20:48:30 (10.7 MB/s) - ‘/content/Brazilian_coins_dataset_ClassSplit.zip’ saved [33540668/33540668]



The zip file is next extracted to the current directory, producing `train` and `validation` subdirectories. In turn each contains `005`, `010`,`025`,`050` and `100` subdirectories for each coin type (value in `centavos` of Brazilian real).

In [ ]:
import os
import zipfile
#extract it. Directories structure as train/val is created
local_zip = '/content/Brazilian_coins_dataset_ClassSplit.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('./')
zip_ref.close()
os.remove('./split_in_folders.c') #C code used to re-arrange the original images in subfolders

Define directories:

In [ ]:
base_dir = './'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

We have a total of 765 training images and 300 validation images. They look like this:

In [ ]:
#@title Show some coins
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
nrows = 4
ncols = 4
# Directory with our training 050 pictures
train_050_dir = os.path.join(train_dir, '050')
# Directory with our training 100 pictures
train_100_dir = os.path.join(train_dir, '100')
train_100_fnames = os.listdir(train_100_dir)
train_050_fnames = os.listdir(train_050_dir)
pic_index = 0 # Index for iterating over images
# Set up matplotlib fig, and size it to fit 4x4 pics
fig = plt.gcf()
fig.set_size_inches(ncols * 4, nrows * 4)
pic_index += 8
next_100_pix = [os.path.join(train_100_dir, fname)
                for fname in train_100_fnames[pic_index-8:pic_index]]
next_050_pix = [os.path.join(train_050_dir, fname)
                for fname in train_050_fnames[pic_index-8:pic_index]]
for i, img_path in enumerate(next_100_pix+next_050_pix):
  # Set up subplot; subplot indices start at 1
  sp = plt.subplot(nrows, ncols, i + 1)
  sp.axis('Off') # Don't show axes (or gridlines)
  img = mpimg.imread(img_path)
  plt.imshow(img)
plt.show()

## Building a Small Convnet from Scratch

Resize all images to 100x100 pixels, so that will be the input size to the neural network.<p>
Define a **shallow** network (there are few training examples, easy to overfit if too much capacity).<p>
Notice that the actual purpose of the Conv layers is **extracting features to classify some coins** (we'll then do transfer learning for our UK coins). Define the final fully-connected layers in a separate step.<p>
# From Keras.io
You might wish to add regularization to some layers as well, to do so plesae check: https://keras.io/api/layers/regularizers/
Regularizers allow you to apply penalties on layer parameters or layer activity during optimization. These penalties are summed into the loss function that the network optimizes.

Regularization penalties are applied on a per-layer basis. The exact API will depend on the layer, but many layers (e.g. Dense, Conv1D, Conv2D and Conv3D) have a unified API.

These layers expose 3 keyword arguments:

    kernel_regularizer: Regularizer to apply a penalty on the layer's kernel
    bias_regularizer: Regularizer to apply a penalty on the layer's bias
    activity_regularizer: Regularizer to apply a penalty on the layer's output


# Clear Keras backend to try new model (Only do this if you want to change model parameters)



In [ ]:
import tensorflow as tf
from tensorflow.keras import backend as K

# Clear the current TensorFlow graph
K.clear_session()

# Reinitialize your model from scratch
model = None  # Explicitly set to None

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras import regularizers #for regularization

In [ ]:
# Input feature map is 100x100x3 colour channels
# img_input = layers.Input(shape=(100, 100, 3))
# #ADD INTERMEDIATE CONV + POOLING LAYERS
# #DON'T ADD FINAL LAYERS AT THIS STEP, USE THE NEXT ONE

img_input = layers.Input(shape=(100, 100, 3))
# Keep the training on only two layers to extract only generalized features like shapes and edges and such, prevent overfitting

x = layers.Conv2D(16, (5, 5), activation='relu')(img_input) # first layer takes image in, finds features (16 filters/kernels)
x = layers.MaxPooling2D((2, 2))(x)

x = layers.Conv2D(32, (3, 3), activation='relu')(x)         # second layer looks at the output of first layer (32 filters)
x = layers.MaxPooling2D((2, 2))(x)
'''
x = layers.Conv2D(64, (3, 3), activation='relu')(x)         # 3rd layer as above 64 filters
x = layers.MaxPooling2D((2, 2))(x)

x = layers.Conv2D(128, (3, 3), activation='relu')(x)         # 4th layer as above 128 filters
x = layers.MaxPooling2D((2, 2))(x)

'''

"\nx = layers.Conv2D(64, (3, 3), activation='relu')(x)         # 3rd layer as above 64 filters\nx = layers.MaxPooling2D((2, 2))(x)\n\nx = layers.Conv2D(128, (3, 3), activation='relu')(x)         # 4th layer as above 128 filters\nx = layers.MaxPooling2D((2, 2))(x)\n\n"

Now **flatten** the feature map and add a couple dense (fully connected) layers. The final one needs **5 outputs** as we're doing a 5-class classification.<p>
<u>We can no longer use a sigmoid output</u> (that handles only 2 classes). See aother vailable activations in https://keras.io/api/layers/activations/
You can add regularization, and also dropout right before the final layer to improve generalization (reduce overfitting)

In [ ]:
# # Flatten feature map to a 1-dim tensor so we can add fully connected layers
# #update x with the output name of the previous step if different
# x = layers.Flatten()(x) # modify accordingly
# # ADD HIDDEN AND FINAL DENSE LAYERS
# # OUTPUT LAYERS NEEDS 5 NODES (one per class). 'sigmoid' ACTIVATION NO LONGER SUITABLE
# # Create model:
# model = Model(img_input, output)

x = layers.Flatten()(x)
x = layers.Dense(64, activation='relu')(x) # reduce numper of neurons due to small network
output = layers.Dense(5, activation='softmax')(x)

# Create model:
model = Model(img_input, output)


Let's summarize the model architecture. NOTICE THE LAYER NAMES, you'll need them later on:

In [ ]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 100, 100, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 96, 96, 16)          │           1,216 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 48, 48, 16)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 46, 46, 32)          │           4,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 23, 23, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 16928)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 64)                  │       1,083,456 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 5)                   │             325 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,089,637 (4.16 MB)

 Trainable params: 1,089,637 (4.16 MB)

 Non-trainable params: 0 (0.00 B)

Next, we'll configure the specifications for model training, using Keras [model.compile](https://keras.io/api/models/model_training_apis/#compile-method) method.<p>
Don't train our model with the `binary_crossentropy` loss, because the final activation cannot be a sigmoid. Check https://keras.io/api/losses/ for alternative **loss** metrics suitable for your new activation.<p>
We can use RMSprop optimizer algorithm `rmsprop`, learning rate commonly 0.001. During training, we will want to monitor classification accuracy. Other available optimizers: https://keras.io/api/optimizers/

In [ ]:
from tensorflow.keras.optimizers import RMSprop
# use categorical crossentropy losss
model.compile(loss='categorical_crossentropy', # Multi-class classification
              optimizer=RMSprop(learning_rate=0.001),
              metrics=['acc'])

### Data Preprocessing

Let's set up data generators that will read pictures in our source folders, convert them to `float32` tensors, and feed them (with their labels 0, 1, 2, ... corresponding to each of the supplied subdirectories) to our network. As our data is split in train and validation folders, we need one generator for the training images and one for the validation images.<p>
Our generators will yield batches of B images and their labels.<p>
Preprocess the images by normalizing the pixel values to be in the `[0.0, 1.0]` range (dividing by 255.0). We can also benefit of **data augmentation** to account for more variability. Rotations are really important as new coins may appear rotated during future inference.<p>
In Keras this can be done via the `keras.preprocessing.image.ImageDataGenerator` class. To find out more augmentation options, check https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator. <p>
We also need to set class_mode to `categorical` (no longer `binary`)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# All images will be rescaled by 1./255 and augmented
train_datagen = ImageDataGenerator(
      rescale=1./255,           # Artificially increase the dataset by modifying the training data as belo:
      rotation_range=40,        #   Allow the NN to train on coins that are at different angles
      width_shift_range=0.2,    #   This helps the model learn to recognize coins even if they are not perfectly centered in the image.
      height_shift_range=0.2,   #   As above but vertically
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')      # fill empty pixels created after the shifts above with a copy of nearest pixel
val_datagen = ImageDataGenerator(rescale=1./255) #do not augment validation
B = 15 #Batch size
# Flow training images in batches of B using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        train_dir,  # This is the source directory for training images
        target_size = (100, 100),  # All images will be resized to 100x100
        batch_size=B,
        # To use categorical_crossentropy loss, we need categorical labels
        class_mode='categorical')
# Flow validation images in batches of 20 using val_datagen generator
validation_generator = val_datagen.flow_from_directory(
        validation_dir,
        target_size=(100, 100),
        batch_size=B,
        class_mode='categorical')

Found 765 images belonging to 5 classes.
Found 300 images belonging to 5 classes.


### Training
Let's train on all 765 images available, for 15 epochs, and validate on all 300 validation images. (This may take a few minutes to run).<p>
If the training accuracy is significantly better than validation accuracy, the model is overfitting, you should include regularization and dropout.


In [ ]:
history = model.fit(
      train_generator,
      steps_per_epoch=765//B,  # 765 images = batch_size * steps
      epochs=20,
      validation_data=validation_generator,
      validation_steps=300//B,  # 300 images = batch_size * steps
      verbose=2)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
51/51 - 8s - 156ms/step - acc: 0.3020 - loss: 1.6587 - val_acc: 0.4333 - val_loss: 1.4027
Epoch 2/20
51/51 - 6s - 117ms/step - acc: 0.4340 - loss: 1.4040 - val_acc: 0.2300 - val_loss: 1.9767
Epoch 3/20
51/51 - 5s - 104ms/step - acc: 0.4954 - loss: 1.2524 - val_acc: 0.5133 - val_loss: 1.0745
Epoch 4/20
51/51 - 5s - 95ms/step - acc: 0.5190 - loss: 1.0940 - val_acc: 0.7233 - val_loss: 0.8702
Epoch 5/20
51/51 - 6s - 117ms/step - acc: 0.6013 - loss: 0.9910 - val_acc: 0.6633 - val_loss: 0.7251
Epoch 6/20
51/51 - 4s - 82ms/step - acc: 0.6052 - loss: 0.9400 - val_acc: 0.6967 - val_loss: 0.7808
Epoch 7/20
51/51 - 3s - 55ms/step - acc: 0.6288 - loss: 0.8739 - val_acc: 0.6900 - val_loss: 0.7483
Epoch 8/20
51/51 - 3s - 68ms/step - acc: 0.6353 - loss: 0.8467 - val_acc: 0.7567 - val_loss: 0.6416
Epoch 9/20
51/51 - 4s - 87ms/step - acc: 0.6889 - loss: 0.7677 - val_acc: 0.6600 - val_loss: 0.9612
Epoch 10/20
51/51 - 3s - 55ms/step - acc: 0.6771 - loss: 0.7764 - val_acc: 0.6867 - val_loss: 0.

---

#Now let's transfer this to our UK coins
We have a very limited collecton of UK coin examples (slightly above 300) split in 8 subdirectories, one for each  class. Let's apply transfer learning from our Brazilian coins.

In [ ]:
!wget https://edshare.gcu.ac.uk/id/document/61325 \
      -O /content/UK_coins_ClassSplit.zip

--2025-03-25 21:09:09--  https://edshare.gcu.ac.uk/id/document/61325
Resolving edshare.gcu.ac.uk (edshare.gcu.ac.uk)... 46.22.140.159
Connecting to edshare.gcu.ac.uk (edshare.gcu.ac.uk)|46.22.140.159|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://edshare.gcu.ac.uk/9959/1/UK_coins_Mario_ClassSplit.zip [following]
--2025-03-25 21:09:10--  https://edshare.gcu.ac.uk/9959/1/UK_coins_Mario_ClassSplit.zip
Reusing existing connection to edshare.gcu.ac.uk:443.
HTTP request sent, awaiting response... 200 OK
Length: 2212692 (2.1M) [application/zip]
Saving to: ‘/content/UK_coins_ClassSplit.zip’

/content/UK_coins_C 100%[===================>]   2.11M  1.83MB/s    in 1.2s    

2025-03-25 21:09:12 (1.83 MB/s) - ‘/content/UK_coins_ClassSplit.zip’ saved [2212692/2212692]



In [ ]:
local_zip = '/content/UK_coins_ClassSplit.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('./')
zip_ref.close()
UK_base_dir = './UK_coins_Mario_ClassSplit'

Take a look to some UK coin images:

In [ ]:
#@title Show some coins
UK_100_dir = os.path.join(UK_base_dir, '100')
UK_050_dir = os.path.join(UK_base_dir, '050')
nrows = 4
ncols = 4
train_100_fnames = os.listdir(UK_100_dir)
train_050_fnames = os.listdir(UK_050_dir)
pic_index = 0 # Index for iterating over images
# Set up matplotlib fig, and size it to fit 4x4 pics
fig = plt.gcf()
fig.set_size_inches(ncols * 4, nrows * 4)
pic_index += 8
next_100_pix = [os.path.join(UK_100_dir, fname)
                for fname in train_100_fnames[pic_index-8:pic_index]]
next_050_pix = [os.path.join(UK_050_dir, fname)
                for fname in train_050_fnames[pic_index-8:pic_index]]
for i, img_path in enumerate(next_100_pix+next_050_pix):
  # Set up subplot; subplot indices start at 1
  sp = plt.subplot(nrows, ncols, i + 1)
  sp.axis('Off') # Don't show axes (or gridlines)
  img = mpimg.imread(img_path)
  plt.imshow(img)
plt.show()

Load the training data. Notice that this time the data is not pre-split in "train" and "validation" directories. We can do an automatic train/validation split as determined by parameter `validation_split`.<p>

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# All images will be rescaled by 1./255 and augmented
UK_datagen = ImageDataGenerator(rescale=1./255,
                                rotation_range=90, #degrees
                                validation_split=0.2)  #makes random split, 20% for validation
B = 10 #Batch size
# Extract flow training images in batches of B images
UK_train_generator = UK_datagen.flow_from_directory(
        UK_base_dir,  # This is the source directory for training images
        target_size = (100, 100),  # All images will be resized to 100x100
        batch_size=B,
        subset = 'training',
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='categorical')
# Extract flow validation images in batches of B images
UK_validation_generator = UK_datagen.flow_from_directory(
        UK_base_dir,
        target_size=(100, 100),
        batch_size=B,
        subset = 'validation',
        class_mode='categorical')

Found 250 images belonging to 8 classes.
Found 59 images belonging to 8 classes.


Now we freeze all layers of our Brazilian CNN, so they don't get re-trained:

In [ ]:
for layer in model.layers:
  print('freezing ' + layer.name)
  layer.trainable = False

freezing input_layer
freezing conv2d
freezing max_pooling2d
freezing conv2d_1
freezing max_pooling2d_1
freezing flatten
freezing dense
freezing dense_1


Next, replace the last FC layers (right after flattening) by new ones. **Output now needs 8 classes!**<p>
Check the layer names to select the desired replacement point.

In [ ]:
replace_point = model.get_layer('flatten')
x = replace_point.output
#x = layers.Dense(128, activation='relu')(x) # 256 might be too many neurons
x = layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x) # reduce from 128
x = layers.Dropout(0.6)(x)                  #  prevent overfitting with a dropout
UKoutput = layers.Dense(8, activation='softmax')(x)

UKmodel = Model(img_input, UKoutput)

Get a summary of the updated  UK model. Is should show a number of non-trainable parameters (those from the frozen layers). The transferred layers should have the same names they had in the original model, added layers will get new names.

In [ ]:
UKmodel.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 100, 100, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 96, 96, 16)          │           1,216 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 48, 48, 16)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 46, 46, 32)          │           4,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 23, 23, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 16928)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 64)                  │       1,083,456 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 8)                   │             520 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,089,832 (4.16 MB)

 Trainable params: 1,083,976 (4.14 MB)

 Non-trainable params: 5,856 (22.88 KB)

Compile the new model. Remember that we need to use a reduced learning rate (like 1/10 of the original one)

In [ ]:
UKmodel.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(learning_rate=0.0001), # Reduced learning rate
              metrics=['acc'])

Train the added layers on our UK coins. We are assuming that the Brazilian coin features extracted by the frozen layers will also be useful for the UK coins...

In [ ]:
history = UKmodel.fit(
      UK_train_generator,
      steps_per_epoch=250//B,  # 250 train images = batch_size * steps
      epochs=40,
      validation_data=UK_validation_generator,
      validation_steps=59//B,  # 59 validation images = batch_size * steps
      verbose=2)

Epoch 1/40
25/25 - 4s - 166ms/step - acc: 0.2000 - loss: 2.1183 - val_acc: 0.4000 - val_loss: 1.9884
Epoch 2/40
25/25 - 1s - 34ms/step - acc: 0.2880 - loss: 1.9580 - val_acc: 0.4200 - val_loss: 1.8445
Epoch 3/40
25/25 - 1s - 34ms/step - acc: 0.3240 - loss: 1.8484 - val_acc: 0.4600 - val_loss: 1.7621
Epoch 4/40
25/25 - 1s - 33ms/step - acc: 0.3840 - loss: 1.7704 - val_acc: 0.4600 - val_loss: 1.7548
Epoch 5/40
25/25 - 1s - 33ms/step - acc: 0.3200 - loss: 1.7696 - val_acc: 0.4000 - val_loss: 1.7044
Epoch 6/40
25/25 - 1s - 33ms/step - acc: 0.3080 - loss: 1.7616 - val_acc: 0.4400 - val_loss: 1.6269
Epoch 7/40
25/25 - 1s - 33ms/step - acc: 0.3760 - loss: 1.6574 - val_acc: 0.4200 - val_loss: 1.5609
Epoch 8/40
25/25 - 1s - 50ms/step - acc: 0.3600 - loss: 1.6339 - val_acc: 0.4600 - val_loss: 1.5353
Epoch 9/40
25/25 - 1s - 33ms/step - acc: 0.4360 - loss: 1.5802 - val_acc: 0.4600 - val_loss: 1.5085
Epoch 10/40
25/25 - 1s - 33ms/step - acc: 0.4120 - loss: 1.5492 - val_acc: 0.4600 - val_loss: 1.442

You might try to improve a bit by unfreezing the last imported Conv layer as well, and retrain some more with a heavily reduced learning rate (1/100 of the original)<p>
Check the layers names and update them as needed.

In [ ]:
unfreeze = False
for layer in UKmodel.layers:
  if layer.name == 'conv2d_1': #first layer to unfreeze
    print('--- unfreezing ---')
    unfreeze = True
  if unfreeze:
    layer.trainable = True
  print(layer.name)

input_layer
conv2d
max_pooling2d
--- unfreezing ---
conv2d_1
max_pooling2d_1
flatten
dense_2
dropout
dense_3


In [ ]:
#show model again. Compare thenumber of non-trainable parameters with previous versions
UKmodel.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 100, 100, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 96, 96, 16)          │           1,216 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 48, 48, 16)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 46, 46, 32)          │           4,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 23, 23, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 16928)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 64)                  │       1,083,456 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 8)                   │             520 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,173,810 (8.29 MB)

 Trainable params: 1,088,616 (4.15 MB)

 Non-trainable params: 1,216 (4.75 KB)

 Optimizer params: 1,083,978 (4.14 MB)

Finally, fine-tune with Stochastic Gradient Descent with very low learning rate for 50 epochs

In [ ]:
# As an optimizer, here we will use SGD
# with a very low learning rate (0.00001)
from tensorflow.keras.optimizers import SGD
UKmodel.compile(loss='categorical_crossentropy',
                optimizer=SGD(learning_rate=0.001/100, momentum=0.9),
                metrics=['acc'])

In [ ]:
history = UKmodel.fit(
      UK_train_generator,
      steps_per_epoch=250//B,  # 221 images = batch_size * steps
      epochs=50,
      validation_data=UK_validation_generator,
      validation_steps=59//B,  # 88 images = batch_size * steps
      verbose=2)

Epoch 1/50
25/25 - 3s - 129ms/step - acc: 0.6240 - loss: 1.1464 - val_acc: 0.5800 - val_loss: 1.1673
Epoch 2/50
25/25 - 1s - 36ms/step - acc: 0.5920 - loss: 1.1123 - val_acc: 0.6200 - val_loss: 1.1264
Epoch 3/50
25/25 - 1s - 50ms/step - acc: 0.6080 - loss: 1.1562 - val_acc: 0.6200 - val_loss: 1.1136
Epoch 4/50
25/25 - 1s - 34ms/step - acc: 0.5880 - loss: 1.1554 - val_acc: 0.6800 - val_loss: 1.1001
Epoch 5/50
25/25 - 1s - 54ms/step - acc: 0.6280 - loss: 1.0905 - val_acc: 0.6600 - val_loss: 1.0894
Epoch 6/50
25/25 - 2s - 81ms/step - acc: 0.5680 - loss: 1.1072 - val_acc: 0.6000 - val_loss: 1.1392
Epoch 7/50
25/25 - 1s - 49ms/step - acc: 0.5720 - loss: 1.1956 - val_acc: 0.6600 - val_loss: 1.1317
Epoch 8/50
25/25 - 1s - 51ms/step - acc: 0.5640 - loss: 1.1749 - val_acc: 0.5600 - val_loss: 1.1973
Epoch 9/50
25/25 - 1s - 34ms/step - acc: 0.5800 - loss: 1.1354 - val_acc: 0.6600 - val_loss: 1.1254
Epoch 10/50
25/25 - 1s - 33ms/step - acc: 0.5840 - loss: 1.1450 - val_acc: 0.6000 - val_loss: 1.155

# Inference
That should be the classifier ready. If you wish, try to play inference on a new image using UKmodel.predict
https://www.tensorflow.org/api_docs/python/tf/keras/Model#predict

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array, load_img
#upload a new coin image and try to classify it
fname = "./200_hard_065.jpg" #update name as needed
img = load_img(fname, target_size=(100, 100))
x = img_to_array(img)  # Numpy array with shape (100, 100, 3)
x = x.reshape((1,) + x.shape)  # Numpy array with shape (1, 100, 100, 3)
# Rescale by 1/255
x /= 255
# Let's run our image through our network
prediction = UKmodel.predict(x)
prediction
#class numbers corresponds to subdirectories (train_datagen.flow_from_directory uses alphabetic order by default)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step


array([[0.00181983, 0.00377008, 0.00803804, 0.00350178, 0.00106717,
        0.01290081, 0.19587043, 0.7730319 ]], dtype=float32)

In [ ]:
# First, print out the class indices to understand which index corresponds to which coin value:
print(UK_train_generator.class_indices)
# Find the index of the highest confidence prediction
predicted_class_index = np.argmax(prediction)
print(f"Predicted class index: {predicted_class_index}")
# Convert the raw prediction to a more readable format:
print("Prediction probabilities:")
for class_name, prob in zip(UK_train_generator.class_indices.keys(), prediction[0]):
    print(f"{class_name}: {prob:.4f}")

{'001': 0, '002': 1, '005': 2, '010': 3, '020': 4, '050': 5, '100': 6, '200': 7}
Predicted class index: 7
Prediction probabilities:
001: 0.0018
002: 0.0038
005: 0.0080
010: 0.0035
020: 0.0011
050: 0.0129
100: 0.1959
200: 0.7730


# Following code is for Pool Size calculation

In [ ]:
# pool size calculation
# Initial input size
input_size = 100  # original input image size

# For a 3x3 Conv2D with no padding
def calculate_conv_output(input_size, kernel_size=3, stride=1):
    return (input_size - kernel_size + 1) // stride

# For a 2x2 MaxPooling2D
def calculate_pool_output(input_size, pool_size=2, stride=2):
    return input_size // pool_size

# Example calculation
current_size = 100
print("Starting size:", current_size)

# First Conv2D + MaxPool
current_size = calculate_pool_output(calculate_conv_output(current_size, kernel_size=5))
print("After first layer:", current_size)

# Second Conv2D + MaxPool
current_size = calculate_pool_output(calculate_conv_output(current_size))
print("After second layer:", current_size)

# 3rd Conv2D + MaxPool
current_size = calculate_pool_output(calculate_conv_output(current_size))
print("After 3rd layer:", current_size)

# 4th Conv2D + MaxPool
current_size = calculate_pool_output(calculate_conv_output(current_size))
print("After 4th layer:", current_size)

Starting size: 100
After first layer: 48
After second layer: 23
After 3rd layer: 10
After 4th layer: 4
